# 🌳 Decision Tree Classifier (Python Implementation – No Libraries)

## 📌 What is a Decision Tree?
A decision tree is a flowchart-like structure that helps make decisions based on input features.  
It splits the data into branches based on conditions until a final decision (class label) is reached.

---

## ✅ Why Use a Decision Tree?

| Reason | Description |
|--------|-------------|
| 🔍 Easy to Understand | Works like human decisions: "If condition, then result" |
| 🧠 No Need for Math Knowledge | Doesn't require linear algebra or calculus |
| 🔀 Handles Mixed Data | Works with both numerical and categorical features |
| 💡 Feature Selection Built-in | Automatically picks important features |
| 📊 Works for Classification & Regression | Predict categories or numeric values |

---

## 💼 What Can It Solve?

- Spam detection (Yes/No)  
- Loan approval (Approve/Reject)  
- Disease prediction (Positive/Negative)  
- Price prediction (in case of regression)  
- Weather-based decisions (like our example)

---

## 🔁 Working Step-by-Step

1. **Calculate Entropy** – Measures impurity in the data.  
2. **Find Information Gain** – Best feature to split on.  
3. **Split the data** based on feature values.  
4. **Repeat recursively** for each branch.  
5. **Stop** when data is pure or no features left.  
6. **Build the tree structure** in nested dictionaries.


In [49]:
import math
from collections import Counter



data = [
    ['Sunny', 'Hot', 'No'],
    ['Sunny', 'Hot', 'No'],
    ['Sunny', 'Mild', 'Yes'],
    ['Sunny', 'Cool', 'Yes'],
    ['Overcast', 'Hot', 'Yes'],
    ['Overcast', 'Mild', 'Yes'],
    ['Overcast', 'Cool', 'Yes'],
    ['Rainy', 'Hot', 'No'],
    ['Rainy', 'Mild', 'Yes'],
    ['Rainy', 'Cool', 'Yes'],
    ['Sunny', 'Mild', 'Yes'],
    ['Rainy', 'Mild', 'Yes'],
    ['Overcast', 'Mild', 'Yes'],
    ['Rainy', 'Hot', 'No'],
    ['Sunny', 'Cool', 'Yes']
]
features = ['Outlook', 'Temp']


## 🧠 Python Code (With Explanation)

### 1. 📌 Entropy Function


In [50]:
def entropy(data):
    labels = [row[-1] for row in data]  # last column = class label
    total = len(labels)
    label_counts = Counter(labels)

    ent = 0.0
    step_by_step = []

    for label in label_counts:
        p = label_counts[label] / total
        ent_component = -p * math.log2(p)
        step_by_step.append(f"Label = {label}, Count = {label_counts[label]}, "
                            f"p = {p:.2f}, -p*log2(p) = {ent_component:.4f}")
        ent += ent_component

    return ent, step_by_step

#### ✔ Purpose:

Calculates how mixed the labels are.  
Entropy is highest when data is 50-50 between "Yes" and "No".  
Lower entropy means purer data.


# What is Entropy?

Entropy measures uncertainty or impurity in the data.

## Formula for Entropy

Let \( H \) be entropy with classes \( c \):

$$
H = - \sum_{i=1}^{c} p_i \log_2 (p_i)
$$

where \( p_i \) is the probability of class \( i \).

## Example Calculation

Suppose our dataset labels are:

| Label | Count | Probability \( p \) |
|-------|-------|---------------------|
| Yes   | 10    | \( \frac{10}{15} = 0.67 \) |
| No    | 5     | \( \frac{5}{15} = 0.33 \)  |

Entropy:

$$
H = - (0.67 \times \log_2 0.67) - (0.33 \times \log_2 0.33) \approx 0.92
$$


### Display Entropy of whole dataset

In [51]:
entropy_value, entropy_steps = entropy(data)
print("### Entropy of entire dataset:")
print(f"Entropy = {entropy_value:.4f}\n")
print("Step-by-step calculation:")
for step in entropy_steps:
    print(step)
# print("\n" + "="*50 + "\n")

### Entropy of entire dataset:
Entropy = 0.8366

Step-by-step calculation:
Label = No, Count = 4, p = 0.27, -p*log2(p) = 0.5085
Label = Yes, Count = 11, p = 0.73, -p*log2(p) = 0.3281


### 2. 📌 Split Data Function

In [52]:
def split_data(data, column, value):
    split = []
    for row in data:
        if row[column] == value:
            reduced_row = row[:column] + row[column+1:]  # remove split feature
            split.append(reduced_row)
    return split


#### ✔ Purpose:

Filters the data where the given column equals the value  
and removes that column from the row (so we don’t use it again).


# What does it do?

- Filters rows where `data[row][column] == value`.
- Removes the split feature column (because it's already used).
- Returns the subset for further splits.

**Example: Split where Outlook = 'Sunny'**

- Original row: `['Sunny', 'Hot', 'No']`
- Split removes 'Sunny' column → `['Hot', 'No']`


### 3. 📌 . Information Gain and Best Split


In [53]:
def best_split(data):
    base_entropy, _ = entropy(data)
    best_gain = 0
    best_column = -1
    n_features = len(data[0]) - 1

    print("### Information Gain Calculation for each feature:")
    for col in range(n_features):
        values = set(row[col] for row in data)
        new_entropy = 0

        print(f"\n- Evaluating feature '{features[col]}':")
        for val in values:
            subset = split_data(data, col, val)
            p = len(subset) / len(data)
            subset_entropy, _ = entropy(subset)
            weighted_entropy = p * subset_entropy
            new_entropy += weighted_entropy

            print(f"  * Value '{val}': subset size = {len(subset)}, p = {p:.2f}, "
                  f"entropy = {subset_entropy:.4f}, weighted entropy = {weighted_entropy:.4f}")

        info_gain = base_entropy - new_entropy
        print(f"Information Gain for '{features[col]}': {info_gain:.4f}")

        if info_gain > best_gain:
            best_gain = info_gain
            best_column = col

    print(f"\nBest feature to split on: '{features[best_column]}' with info gain = {best_gain:.4f}\n")
    return best_column


#### ✔ Purpose:
Finds the best feature (column) to split on by calculating information gain for each one. Higher info gain = better split.

# What is Information Gain?

Information Gain is the reduction in entropy due to splitting on a feature.

$$
IG(S, A) = H(S) - \sum_{v \in \text{Values}(A)} \frac{|S_v|}{|S|} H(S_v)
$$

- \( H(S) \): entropy of the full dataset.  
- \( S_v \): subset of data where feature \( A \) has value \( v \).  
- \( \frac{|S_v|}{|S|} \): proportion of subset size.


---

## Step-by-step:

For each feature (column):

1. Calculate unique values of that feature.
2. For each value:
   - Create subset with that value.
   - Calculate entropy of subset.
   - Calculate weighted entropy = subset size ratio \(\times\) subset entropy.
3. Sum weighted entropy of all subsets.
4. Calculate information gain = base entropy − weighted entropy sum.
5. Track the feature with max information gain.


### 4. 📌 Build Tree Function

In [54]:
def build_tree(data, features):
    labels = [row[-1] for row in data]

    if labels.count(labels[0]) == len(labels):
        return labels[0]

    if len(data[0]) == 1:
        return Counter(labels).most_common(1)[0][0]

    best_feat = best_split(data)
    best_feat_name = features[best_feat]

    tree = {best_feat_name: {}}
    values = set(row[best_feat] for row in data)

    for val in values:
        subset = split_data(data, best_feat, val)
        sub_features = features[:best_feat] + features[best_feat+1:]
        subtree = build_tree(subset, sub_features)
        tree[best_feat_name][val] = subtree

    return tree


#### ✔ Purpose:

Builds the decision tree recursively:

- **Base case:** If all labels are same → return result.  
- Otherwise, find best feature, split data, and build branches.  
- Tree is stored as a dictionary.


# Logic

- If all labels are the same, return the label (leaf node).
- If no features left, return majority label.
- Else:
  1. Find best feature to split on.
  2. For each value of that feature:
     - Split dataset.
     - Recursively build subtree.
  3. Return tree node as dict.


### 5. 📌 Run & Output Final Tree

In [55]:
decision_tree = build_tree(data, features)
print("### Final Decision Tree:")
print(decision_tree)


### Information Gain Calculation for each feature:

- Evaluating feature 'Outlook':
  * Value 'Overcast': subset size = 4, p = 0.27, entropy = 0.0000, weighted entropy = 0.0000
  * Value 'Sunny': subset size = 6, p = 0.40, entropy = 0.9183, weighted entropy = 0.3673
  * Value 'Rainy': subset size = 5, p = 0.33, entropy = 0.9710, weighted entropy = 0.3237
Information Gain for 'Outlook': 0.1457

- Evaluating feature 'Temp':
  * Value 'Cool': subset size = 4, p = 0.27, entropy = 0.0000, weighted entropy = 0.0000
  * Value 'Hot': subset size = 5, p = 0.33, entropy = 0.7219, weighted entropy = 0.2406
  * Value 'Mild': subset size = 6, p = 0.40, entropy = 0.0000, weighted entropy = 0.0000
Information Gain for 'Temp': 0.5960

Best feature to split on: 'Temp' with info gain = 0.5960

### Information Gain Calculation for each feature:

- Evaluating feature 'Outlook':
  * Value 'Overcast': subset size = 1, p = 0.20, entropy = 0.0000, weighted entropy = 0.0000
  * Value 'Sunny': subset size = 2, 

#### ✔ Output:
Will print a dictionary representing the final decision tree structure.

In [41]:
# pip install graphviz

from graphviz import Digraph

def add_nodes_edges(tree, dot=None, parent=None):
    if dot is None:
        dot = Digraph()
        dot.attr('node', shape='ellipse', fontsize='10')

    for node, branches in tree.items():
        # Add current node
        dot.node(node)

        if parent:
            dot.edge(parent, node)

        if isinstance(branches, dict):
            for edge_label, subtree in branches.items():
                # Create a unique id for subtree nodes to avoid collisions
                if isinstance(subtree, dict):
                    # subtree is another dict - create new node with a random name
                    sub_node = f"{node}_{edge_label}"
                    dot.node(sub_node, label=edge_label)
                    dot.edge(node, sub_node, label=edge_label)
                    # Recursive call
                    add_nodes_edges(subtree, dot, sub_node)
                else:
                    # Leaf node
                    leaf_name = f"{node}_{edge_label}_{subtree}"
                    dot.node(leaf_name, label=subtree, shape='box', style='filled', color='lightgrey')
                    dot.edge(node, leaf_name, label=edge_label)
        else:
            # Leaf node (should not occur here usually)
            dot.node(branches)
            if parent:
                dot.edge(parent, branches)

    return dot

# Use this function on your tree:

dot = add_nodes_edges(decision_tree)
dot.render('decision_tree_graph', view=True)  # This saves and opens the graph PDF file


'decision_tree_graph.pdf'

# 📌 What is a Random Forest?

A Random Forest is an ensemble of many decision trees working together to make a better, more accurate prediction.

Instead of relying on one decision tree, it builds multiple trees on random samples of the data and features, then combines their results by voting.

---

## ✅ Why Use a Random Forest?

| Reason            | Description                                                                                  |
|-------------------|----------------------------------------------------------------------------------------------|
| 🌲 Multiple Trees  | Reduces the chance of overfitting (too much fitting to training data) by averaging many trees. |
| 🎲 Randomness      | Each tree trains on a random subset of data and features, adding diversity.                   |
| 🤝 Better Accuracy | Combining many trees usually gives better predictions than a single tree.                     |
| 💪 Handles Overfitting | More robust than a single decision tree, less prone to mistakes on new data.               |
| 👩‍💻 Easy to Use   | Works well with minimal tuning and handles both classification and regression.                |

---

## 💡 What Can It Solve?

- Email spam detection (spam or not)  
- Credit risk evaluation (safe or risky loan)  
- Medical diagnosis (disease or no disease)  
- Stock price prediction (continuous value)  
- Weather forecasting (rain/no rain)  

---

## 🔁 How Does Random Forest Work Step-by-Step?

1. **Bootstrap Sampling:** Create multiple random samples (with replacement) from the training data.  
2. **Build Decision Trees:** For each sample, build a decision tree using a random subset of features at each split.  
3. **Make Predictions:** To predict, each tree gives its own result.  
4. **Majority Voting:** Combine predictions from all trees — the most common prediction wins (for classification).  
5. **Output:** Final prediction is usually more accurate and stable than a single decision tree.  

---

## 📝 Summary

Random Forest = many decision trees + randomness + voting.

It reduces errors and overfitting by averaging multiple trees.

Widely used because it’s simple, effective, and powerful.


In [60]:
import random
import math
from collections import Counter

# 1. Entropy calculation function (same as before)
def entropy(data):
    labels = [row[-1] for row in data]  # Extract class labels (last element in each row)
    total = len(labels)                  # Total number of samples
    counts = Counter(labels)             # Count how many times each label occurs

    ent = 0.0                           # Initialize entropy

    for label in counts:
        p = counts[label] / total       # Probability of label
        ent -= p * math.log2(p)         # Entropy formula: sum(-p*log2(p))
    return ent


# 2. Split data based on feature and value
def split_data(data, column, value):
    split = []
    for row in data:
        if row[column] == value:
            # Remove the feature column because we split on it
            reduced_row = row[:column] + row[column+1:]
            split.append(reduced_row)
    return split


# 3. Build one decision tree (simplified, no pruning)
def build_tree(data, features):
    labels = [row[-1] for row in data]

    if labels.count(labels[0]) == len(labels):
        return labels[0]          # If all labels same, return that label (leaf node)

    if len(data[0]) == 1:
        return Counter(labels).most_common(1)[0][0]  # If no more features, return majority label

    base_entropy = entropy(data)
    best_gain = 0
    best_col = -1
    best_vals = set()

    n_features = len(features)

    # Select random subset of features (half) for randomness (random forest concept)
    features_idx = random.sample(range(n_features), max(1, n_features // 2))

    for col in features_idx:
        values = set(row[col] for row in data)   # Unique values of feature in dataset
        new_entropy = 0

        for val in values:
            subset = split_data(data, col, val)
            p = len(subset) / len(data)          # Weight of subset
            new_entropy += p * entropy(subset)   # Weighted entropy

        info_gain = base_entropy - new_entropy     # Information gain by splitting on feature

        if info_gain > best_gain:
            best_gain = info_gain
            best_col = col
            best_vals = values

    if best_col == -1:
        return Counter(labels).most_common(1)[0][0]  # No good split, return majority label

    tree = {features[best_col]: {}}
    for val in best_vals:
        subset = split_data(data, best_col, val)
        sub_features = features[:best_col] + features[best_col+1:]  # Remove used feature
        tree[features[best_col]][val] = build_tree(subset, sub_features)  # Recursive call

    return tree


# 4. Bootstrap sample (random sample with replacement)
def bootstrap_sample(data):
    n = len(data)
    return [random.choice(data) for _ in range(n)]

# 5. Predict for one tree
def predict(tree, features, sample):
    if not isinstance(tree, dict):
        return tree          # Leaf node reached, return label

    root = next(iter(tree))  # Get root feature to split on
    feature_val = sample[features.index(root)]  # Value of feature in sample
    subtree = tree[root].get(feature_val)       # Traverse down the branch

    if subtree is None:
        return None        # If no branch for value, return None

    # Remove used feature and recurse
    return predict(subtree, [f for f in features if f != root],
                   sample[:features.index(root)] + sample[features.index(root)+1:])

# 6. Random Forest predict - majority vote
def random_forest_predict(trees, features, sample):
    predictions = []
    for tree in trees:
        pred = predict(tree, features, sample)
        if pred is not None:
            predictions.append(pred)

    if predictions:
        # Return most common prediction
        return Counter(predictions).most_common(1)[0][0]
    else:
        return None


# 7. Train random forest
def train_random_forest(data, features, n_trees=5):
    trees = []
    for _ in range(n_trees):
        sample = bootstrap_sample(data)       # Random subset with replacement
        tree = build_tree(sample, features)   # Build one tree
        trees.append(tree)
    return trees


# ====== Example usage ======

data = [
    ['Sunny', 'Hot', 'No'],
    ['Sunny', 'Hot', 'No'],
    ['Sunny', 'Mild', 'Yes'],
    ['Sunny', 'Cool', 'Yes'],
    ['Overcast', 'Hot', 'Yes'],
    ['Overcast', 'Mild', 'Yes'],
    ['Overcast', 'Cool', 'Yes'],
    ['Rainy', 'Hot', 'No'],
    ['Rainy', 'Mild', 'Yes'],
    ['Rainy', 'Cool', 'Yes'],
]

features = ['Outlook', 'Temp']

# Train forest
forest = train_random_forest(data, features, n_trees=5)

# Predict example sample
sample = ['Sunny', 'Cool']
prediction = random_forest_predict(forest, features, sample)
print(f"Prediction for {sample} = {prediction}")


Prediction for ['Sunny', 'Cool'] = Yes


In [57]:
from graphviz import Digraph

def add_nodes_edges(tree, dot=None, parent=None):
    if dot is None:
        dot = Digraph()
        dot.attr('node', shape='box')

    if not isinstance(tree, dict):
        # Leaf node
        node_id = str(id(tree))
        dot.node(node_id, str(tree), shape='ellipse', style='filled', color='lightgrey')
        if parent:
            dot.edge(parent, node_id)
        return dot

    # Internal node
    root = next(iter(tree))
    node_id = str(id(tree))
    dot.node(node_id, root)

    if parent:
        dot.edge(parent, node_id)

    for val, subtree in tree[root].items():
        val_id = f"{node_id}_{val}"
        dot.node(val_id, str(val), shape='oval', style='filled', color='lightblue')
        dot.edge(node_id, val_id)
        add_nodes_edges(subtree, dot, val_id)

    return dot


In [59]:
# Visualize first tree in forest
dot = add_nodes_edges(forest[0])
dot.render('decision_tree', format='pdf', cleanup=True)  # Saves decision_tree.png file
print("Decision tree graph saved as decision_tree.png")


Decision tree graph saved as decision_tree.png
